# LUCID Advanced

In this example, we dive deeper into canonical inverse design by exploring some different configurations. Note that there is a default configuration which works for many exampels. The first part of the code is from the basic tensorflow example.

In [1]:
import pandas as pd
import tensorflow as tf

from canonical_sets.data import Adult
from canonical_sets.models import ClassifierTF
from canonical_sets import LUCID

tf.keras.utils.set_random_seed(42)

data = Adult()

model = ClassifierTF(2)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics = ["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5)

model.fit(data.train_data.to_numpy(), data.train_labels.to_numpy(), epochs=200,
 validation_data=(data.val_data.to_numpy(), data.val_labels.to_numpy()), callbacks=[callback])

model.evaluate(data.test_data.to_numpy(), data.test_labels.to_numpy())

example_data = data.train_data
outputs = pd.DataFrame([[0, 1]], columns=["<=50K", ">50K"])

Epoch 1/200
755/755 [==============================] - 2s 2ms/step - loss: 0.4249 - accuracy: 0.7994 - val_loss: 0.3708 - val_accuracy: 0.8294
Epoch 2/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3649 - accuracy: 0.8265 - val_loss: 0.3563 - val_accuracy: 0.8356
Epoch 3/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3572 - accuracy: 0.8312 - val_loss: 0.3523 - val_accuracy: 0.8376
Epoch 4/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3541 - accuracy: 0.8338 - val_loss: 0.3491 - val_accuracy: 0.8351
Epoch 5/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3523 - accuracy: 0.8345 - val_loss: 0.3471 - val_accuracy: 0.8371
Epoch 6/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3511 - accuracy: 0.8344 - val_loss: 0.3465 - val_accuracy: 0.8369
Epoch 7/200
755/755 [==============================] - 1s 1ms/step - loss: 0.3501 - accuracy: 0.8360 - val_loss: 0.3456 - val_accuracy: 0.8357

Canonical inverse design depends on a few hyperparameters, such as the number of samples, number of epochs, and the learning rate.

In [2]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 13.97it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.924375e-01  0.007562  ...                0                   0
       5      4.237517e-22  1.000000  ...                0                   0
       6      4.276351e-07  1.000000  ...                0                   0
2      1      2.573124e-01  0.742688  ...                0                   0
       5      2.811741e-07  1.000000  ...                0                   0

[5 rows x 106 columns]

You will notice that there are 6 epochs instead of the requested 5 (numb_of_epochs). This is because the extra_epoch arugment is True by default. Later in this tutorial, we will get into more detail, and just put it on False for now.

In [3]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, extra_epoch=False)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 24.28it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.924375e-01  0.007562  ...                0                   0
       5      4.237517e-22  1.000000  ...                0                   0
2      1      2.573124e-01  0.742688  ...                0                   0
       5      2.811741e-07  1.000000  ...                0                   0
3      1      1.750611e-02  0.982494  ...                0                   0

[5 rows x 106 columns]

Besides these hyperparameters, there is also the range of the uniform dsitribution from which we are sampling the initial input vectors.

In [4]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=False)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 23.21it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.113827e-01  0.088617  ...                0                   0
       5      2.751082e-21  1.000000  ...                0                   0
2      1      3.457343e-01  0.654266  ...                0                   0
       5      3.462221e-09  1.000000  ...                0                   0
3      1      1.070129e-01  0.892987  ...                0                   0

[5 rows x 106 columns]

There is also the pre- and post-processing of the categorical features. By randomly drawing values from the uniform distribution, the initial input vectors do not look like real examples with clean one-hot encoded categorical features. By setting one_hot_pre to True, the features are one-hot encoded by setting the max value to 1 and all the others to 0. Note that the default value here is False and thus that the inverse design starts from the randomly drawn values.

In [5]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=False,
 one_hot_pre=True)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 24.87it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      5.788748e-01  0.421125  ...                0                   0
       5      2.729235e-14  1.000000  ...                0                   0
2      1      8.363793e-01  0.163621  ...                0                   0
       5      8.151322e-20  1.000000  ...                0                   0
3      1      5.620487e-01  0.437951  ...                0                   0

[5 rows x 106 columns]

It may seem that the output has hardly changed (but pay attention to the outcomes as they have changed) and to get a better understanding of what is happening we can set one_hot_post to False as it is True by default.

In [6]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=False,
 one_hot_pre=True, one_hot_post=False)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 22.12it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      5.788748e-01  0.421125  ...         0.000000            0.000000
       5      2.729235e-14  1.000000  ...        -0.286929            0.294768
2      1      8.363793e-01  0.163621  ...         0.000000            0.000000
       5      8.151322e-20  1.000000  ...        -0.414565            0.425892
3      1      5.620487e-01  0.437951  ...         0.000000            0.000000

[5 rows x 106 columns]

Looking at the categorical variables, we now see that only the first epoch is one-hot encoded. And setting both to False:

In [7]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=False,
 one_hot_pre=False, one_hot_post=False)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 22.86it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.113827e-01  0.088617  ...        -0.198243           -0.325154
       5      2.751082e-21  1.000000  ...        -0.649984            0.138931
2      1      3.457343e-01  0.654266  ...         0.053227            0.262911
       5      3.462221e-09  1.000000  ...        -0.118142            0.438962
3      1      1.070129e-01  0.892987  ...        -0.273668           -0.337543

[5 rows x 106 columns]

We see that none of them is one-hot encoded. So, the pre-processing happens before the inverse design and thus affects the outcomes, whereas the post-processing is done after the inverse design and does not affect the outcomes. By default, we apply post-processing on all epochs for a clean view, but no pre-processing so we start from the initially random vectors. To see whether the "real" examples at the end would lead to very different conclusions from the "unrealistic" examples that we get from the inverse design we one-hot encode the categorical features and run an additional forward pass. This is done by setting the extra_epoch argument to True again (which it is by default).

In [8]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=True,
 one_hot_pre=False, one_hot_post=False)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 14.34it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.113827e-01  0.088617  ...        -0.198243           -0.325154
       5      2.751082e-21  1.000000  ...        -0.649984            0.138931
       6      7.542913e-07  0.999999  ...         0.000000            0.000000
2      1      3.457343e-01  0.654266  ...         0.053227            0.262911
       5      3.462221e-09  1.000000  ...        -0.118142            0.438962

[5 rows x 106 columns]

The extra epoch is now the only one-hot encoded variable. By applying post-processing again, you will see that the outcomes do not change, but that the categorical features are now just one-hot encoded across all epochs.

In [9]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=True,
 one_hot_pre=False, one_hot_post=True)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 13.95it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.113827e-01  0.088617  ...                0                   0
       5      2.751082e-21  1.000000  ...                0                   0
       6      7.542913e-07  0.999999  ...                0                   0
2      1      3.457343e-01  0.654266  ...                0                   0
       5      3.462221e-09  1.000000  ...                0                   0

[5 rows x 106 columns]

Finally, we have been mostly focusing on the first and last example, but the log_every_n argument allows us to monitor many more examples and to perform a dynamic analysis.

In [10]:
lucid = LUCID(model, outputs, example_data, numb_of_samples=10,
 numb_of_epochs=5, lr=1, low=-0.5, high=0.5, extra_epoch=True,
 one_hot_pre=False, one_hot_post=True, log_every_n=1)
lucid.results.head()

100%|██████████| 10/10 [00:00<00:00, 12.68it/s]


<=50K      >50K  ...  Country+Vietnam  Country+Yugoslavia
sample epoch                          ...                                     
1      1      9.113827e-01  0.088617  ...                0                   0
       2      2.751082e-21  1.000000  ...                0                   0
       3      2.751082e-21  1.000000  ...                0                   0
       4      2.751082e-21  1.000000  ...                0                   0
       5      2.751082e-21  1.000000  ...                0                   0

[5 rows x 106 columns]

Hopefully, canonical inverse design seems less mysterious right now! Note that many of the arguments have default values that apply to many cases and that you are not required to specify all of them all the time. If there are still things unclear, feel free to submit an issue or a PR!